In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics
using Flight.FlightComponents
using Flight.FlightAircraft.C172
using Flight.FlightAircraft.C172FBW
using Flight.FlightAircraft.C172FBW.FlightControl
using Flight.FlightComponents.Control.Discrete: PIDParams
using Flight.FlightComponents.Control.PIDOpt: Settings, Metrics, optimize_PID, build_PID, check_results

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

## 1. Stability Augmentation

We start by taking a look at the linearized lateral model:

In [2]:
global_search = false

ac = Cessna172FBW(NED()) |> System #linearization requires NED kinematics

design_point = C172.TrimParameters()
# design_point = C172.TrimParameters(EAS = 28)

#complete lateral model
lss_lat = Control.Continuous.LinearizedSS(ac, design_point; model = :lat);
P_lat = named_ss(lss_lat);

x_labels_lat = keys(lss_lat.x0) |> collect
y_labels_lat = keys(lss_lat.y0) |> collect
u_labels_lat = keys(lss_lat.u0) |> collect

lss_lat.A[:ψ, :] |> display
lss_lat.A[:, :ψ] |> display
lss_lat.C[:, :ψ] |> display

ComponentVector{Float64}(p = 0.0, r = 1.0002770254545765, ψ = 1.3081580954809074e-9, φ = -8.294400071674642e-6, v_x = -1.1414420617798425e-18, v_y = -3.6927682278764967e-9, β_filt = 0.0, ail_v = 0.0, ail_p = 0.0, rud_v = 0.0, rud_p = 0.0)

ComponentVector{Float64}(p = -5.220765832703034e-7, r = -2.3909657183990296e-6, ψ = 1.3081580954809074e-9, φ = 5.554732779696859e-8, v_x = 0.00018056471393279594, v_y = -7.123678585685411e-6, β_filt = 0.0, ail_v = 0.0, ail_p = 0.0, rud_v = 0.0, rud_p = 0.0)

ComponentVector{Float64}(p = 0.0, r = 0.0, ψ = 1.0, φ = 0.0, v_x = 0.0, v_y = 0.0, β_filt = 0.0, ail_v = 0.0, ail_p = 0.0, rud_v = 0.0, rud_p = 0.0, f_y = -1.1990778148174286e-8, β = 0.0, χ = 0.9999999999995453, aileron_cmd = 0.0, rudder_cmd = 0.0)

As one would expect, heading is nearly a pure integral of yaw rate. This will be a problem when designing a LQR tracker for which the choice of command variables involves non-zero steady state yaw rate. Since we cannot maintain constant heading with an arbitrary yaw rate, true static equilibrium will not exist and the block $H_x F^{-1} G$ in the feedforward gain matrix required for tracking such command variables will be singular. This would not be an issue if we picked for instance $\chi$ and $\beta$ as command variables, because true equilibrium exists for an arbitrary combination of the two. However, this is not an appealing choice, because we would like to be able to control the roll axis explicitly, rather than implicitly through $\chi$.

To handle this issue, we could partition the state and work with quasi-static equilibrium. However, since this is annoying, and the coupling of $\psi$ into the remaining states is extremely weak, we can simply drop $\psi$ from the state, design the LQR trackers on the reduced plant, and then connect them back to the full plant. 

In [3]:
x_labels = copy(x_labels_lat)
y_labels = copy(y_labels_lat)
u_labels = copy(u_labels_lat)

x_labels = deleteat!(x_labels, findfirst(isequal(:ψ), x_labels))
y_labels = deleteat!(y_labels, findfirst(isequal(:ψ), y_labels))

#we should also remove χ because of its coupling to ψ
y_labels = deleteat!(y_labels, findfirst(isequal(:χ), y_labels))

lss_red = Control.Continuous.submodel(lss_lat; x = x_labels, u = u_labels, y = y_labels)
P_red = named_ss(lss_red);

In [4]:
dampreport(P_lat)
dampreport(P_red)
controllability(P_red)

|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+
| +8.87e-06          |  -1           |  8.87e-06     |  1.41e-06     |  -1.13e+05    |
| -0.0214            |  1            |  0.0214       |  0.00341      |  46.7         |
| -0.131             |  1            |  0.131        |  0.0208       |  7.65         |
| -0.686  ±   2.69im |  0.247        |  2.77         |  0.442        |  1.46         |
| -10.5              |  1            |  10.5         |  1.66         |  0.0956       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -50                |  1            |  50           |  7.96         |  0.02         |
|        Pole        |   Damping     |   Fr

(iscontrollable = true, ranks = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10], sigma_min = [0.004495011562276492, 14.173180838514225, 14.173180838514225, 14.173180838514225, 14.173180838514225, 0.8448024030616802, 0.47891324958434894, 0.47891324958434894, 0.09432187369760883, 0.14536969251937806])

We see that removing $\psi$ as a state eliminates an extremely slow pole, probably geodesy-related. The remaining poles are virtually unaffected. Also, it significantly improves the condition of the dynamics matrix:

In [5]:
cond(P_lat.A) |> display
cond(P_red.A) |> display

9.646214034825184e8

101524.66452369448

In [6]:
#ensure consistency in component selection and ordering between our design model
#and FBWv1 avionics implementation for state and control vectors
@assert tuple(x_labels...) === propertynames(FlightControl.XLat())
@assert tuple(u_labels...) === propertynames(FlightControl.ULat())

x_trim = lss_red.x0
u_trim = lss_red.u0

n_x = length(x_labels)
n_u = length(u_labels)

UndefVarError: UndefVarError: `XLat` not defined

### 1.1. Design A

We first test a purely proportional LQR Tracker design. This requires no integral augmentation, so the feedback matrix can be determined directly from the open loop system's matrices.

In [7]:
#useful signal labels for connections
u_labels_fbk = Symbol.(string.(u_labels) .* "_fbk")
u_labels_fwd = Symbol.(string.(u_labels) .* "_fwd")
u_labels_sum = Symbol.(string.(u_labels) .* "_sum")

@unpack v_x, v_y = lss_red.x0
v_norm = norm([v_x, v_y])

#weight matrices
Q = FlightControl.XLat(p = 0, r = 0.1, φ = 0.1, v_x = 0/v_norm, v_y = 0/v_norm, β_filt = 0, ail_v = 0, ail_p = 0, rud_v = 0, rud_p = 0) |> diagm
R = FlightControl.ULat(aileron_cmd = 0.1, rudder_cmd = 0.01) |> diagm

#feedback gain matrix
C_fbk = lqr(P_red, Q, R)
C_fbk_ss = named_ss(ss(C_fbk); u = x_labels, y = u_labels_fbk)

#summing junctions
aileron_sum = sumblock("aileron_cmd_sum = aileron_cmd_fwd- aileron_cmd_fbk")
rudder_sum = sumblock("rudder_cmd_sum = rudder_cmd_fwd - rudder_cmd_fbk")

connections_fbk = vcat(
    Pair.(x_labels, x_labels),
    Pair.(u_labels_fbk, u_labels_fbk),
    Pair.(u_labels_sum, u_labels),
    )

P_fbk = connect([P_red, aileron_sum, rudder_sum, C_fbk_ss], connections_fbk; w1 = u_labels_fwd, z1 = y_labels)

dampreport(P_red)
dampreport(P_fbk)

UndefVarError: UndefVarError: `XLat` not defined

In [8]:
a2a_fbk = P_fbk[:aileron_cmd, :aileron_cmd_fwd]
a2p = P_red[:p, :aileron_cmd]
a2p_fbk = P_fbk[:p, :aileron_cmd_fwd]
a2φ = P_red[:φ, :aileron_cmd]
a2φ_fbk = P_fbk[:φ, :aileron_cmd_fwd]
step(a2p, 10) |> plot
step(a2p_fbk, 10) |> plot!
step(a2a_fbk, 10) |> plot!
step(a2φ_fbk, 10) |> plot!
# step(a2φ, 10) |> plot!

UndefVarError: UndefVarError: `P_fbk` not defined

In [9]:
a2r = P_red[:r, :aileron_cmd]
a2r_fbk = P_fbk[:r, :aileron_cmd_fwd]
step(a2r, 10) |> plot
step(a2r_fbk, 10) |> plot!

UndefVarError: UndefVarError: `P_fbk` not defined

In [10]:
a2β_fbk = P_fbk[:β, :aileron_cmd_fwd]
step(a2β_fbk, 10) |> plot

UndefVarError: UndefVarError: `P_fbk` not defined

In [11]:
a2f_fbk = P_fbk[:f_y, :aileron_cmd_fwd]
step(a2f_fbk, 10) |> plot

UndefVarError: UndefVarError: `P_fbk` not defined

In [12]:
ail2rud_fbk = P_fbk[:rudder_cmd, :aileron_cmd_fwd]
step(ail2rud_fbk, 10) |> plot

UndefVarError: UndefVarError: `P_fbk` not defined

Since bank angle is essentially a pure integral of roll rate, asymptotic equilibrium is only possible for a zero steady-state roll rate. Indeed, the closed-loop system's aileron to roll-rate step response goes to zero in the steady state. This indicates that selecting roll rate as a tracked variable would yield a (nearly) singular feedforward matrix.

For an aileron command step input, aileron command output starts at 1, and then equilibrium is achieved with a small (although not zero) value. This means that aileron command isn't a good choice for a tracked variable either, because achieving unit steady-state output in the regulated system would require very large feedforward gains and would yield an extremely high initial actuator command, well beyond 1 (at which actuator is already saturated).

What variables should then be selected for tracking? The only one we are interested in controlling explicitly is $\beta$ (or alternatively lateral specific force, $f_y$). In most cases we want to maintain $\beta = 0$ for turn coordination, but we might want to command non-zero values for cross-wind landing. If we select $\beta$ as the only command variable, we will get a 2x1 feedforward matrix which will provide a combination of aileron and rudder that achieves the desired steady-state $\beta$ (specifically, the optimal one in the least-squares sense). However, we obviously also want to retain roll control. But if we add an arbitrary external aileron command to that prescribed by the $\beta$ feedforward, we will no longer achieve the desired steady-state $\beta$ value.

Looking at the closed-loop aileron to bank angle response, we see that it starts at zero and then stabilizes at a value close to 1. This suggests that $\phi$ and $\beta$ would make for a good combination of tracked variables. For $\beta$, it is useful to add integral control. Now, once we have this MIMO system with $\phi$ and $\beta$ as inputs, we can add an external SISO loop to command roll rate, with the necessary compensation to achieve zero steady state error.

Now, we determine the feedforward gain matrix for commanding $\varphi$ and $\beta$. Following Stengel section 6.2:

In [13]:
#define command variables
z_labels = [:φ, :β]
@assert tuple(z_labels...) === propertynames(FlightControl.ZLatPhiBeta())
z_labels_sp = Symbol.(string.(z_labels) .* "_sp")

z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)

F = lss_red.A
G = lss_red.B
H_x = lss_red.C[z_labels, :]
H_u = lss_red.D[z_labels, :]

A = [F G; H_x H_u]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]
C_fwd = B_22 + C_fbk * B_12
C_fwd_ss = named_ss(ss(C_fwd), u = z_labels_sp, y = u_labels_fwd)

display(z_labels)
display(H_x)
display(H_u)
display(C_fwd)

UndefVarError: UndefVarError: `ZLatPhiBeta` not defined

In [14]:
connections_fwd = Pair.(C_fwd_ss.y, C_fwd_ss.y)
P_φβ = connect([C_fwd_ss, P_fbk], connections_fwd; w1 = z_labels_sp, z1 = P_fbk.y);

UndefVarError: UndefVarError: `C_fwd_ss` not defined

In [15]:
φsp2φ_SAS = P_φβ[:φ, :φ_sp]
φsp2β_SAS = P_φβ[:β, :φ_sp]
φsp2ail_SAS = P_φβ[:aileron_cmd, :φ_sp]
φsp2rud_SAS = P_φβ[:rudder_cmd, :φ_sp]
step(φsp2φ_SAS, 10) |> plot
step(φsp2ail_SAS, 10) |> plot!
step(φsp2β_SAS, 10) |> plot!
step(φsp2rud_SAS, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

In [16]:
βsp2β_SAS = P_φβ[:β, :β_sp]
βsp2φ_SAS = P_φβ[:φ, :β_sp]
βsp2rud_SAS = P_φβ[:rudder_cmd, :β_sp]
βsp2ail_SAS = P_φβ[:aileron_cmd, :β_sp]
step(βsp2β_SAS, 10) |> plot
step(βsp2φ_SAS, 10) |> plot!
step(βsp2rud_SAS, 10) |> plot!
step(βsp2ail_SAS, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

Beautiful. Now that we know that $\phi$ and $\beta$ work well, we can repeat the design with added integral control in $\beta$.

### 1.2. Design B (Integral $\beta$ Control)

In [17]:
#define command variables
z_labels = [:φ, :β]
@assert tuple(z_labels...) === propertynames(FlightControl.ZLatPhiBeta())
z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)

UndefVarError: UndefVarError: `ZLatPhiBeta` not defined

In [18]:
F = lss_red.A
G = lss_red.B
Hx = lss_red.C[z_labels, :]
Hu = lss_red.D[z_labels, :]
display(Hx)
display(Hu)

#define the blocks corresponding to the subset of the command variables for
#which integral compensation is required. in this case, only one of them (β).
#Since the resulting blocks have only one row, we get vectors, which we need to
#transpose to get the desired row matrices back.
Hx_int = Hx[:β, :]'
Hu_int = Hu[:β, :]'
n_int, _ = size(Hx_int)
display(Hx_int)
display(Hu_int)

F_aug = [F zeros(n_x, n_int); Hx_int zeros(n_int, n_int)]
G_aug = [G; Hu_int]
Hx_aug = [Hx zeros(n_z, n_int)]
Hu_aug = Hu

P_aug = ss(F_aug, G_aug, Hx_aug, Hu_aug)
controllability(F_aug, G_aug)

2×10 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(p = 1, r = 2, φ = 3, v_x = 4, v_y = 5, β_filt = 6, ail_v = 7, ail_p = 8, rud_v = 9, rud_p = 10)
 0.0  0.0  1.0  0.0  0.0        0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0190216  0.0  0.0  0.0  0.0  0.0

2×2 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(aileron_cmd = 1, rudder_cmd = 2)
 0.0  0.0
 0.0  0.0

1×10 adjoint(::ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(p = 1, r = 2, φ = 3, v_x = 4, v_y = 5, β_filt = 6, ail_v = 7, ail_p = 8, rud_v = 9, rud_p = 10)}}}) with eltype Float64 with indices Base.OneTo(1)×1:1:10:
 0.0  0.0  0.0  0.0  0.0190216  0.0  0.0  0.0  0.0  0.0

1×2 adjoint(::ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(aileron_cmd = 1, rudder_cmd = 2)}}}) with eltype Float64 with indices Base.OneTo(1)×1:1:2:
 0.0  0.0

UndefVarError: UndefVarError: `n_x` not defined

In [19]:
@unpack v_x, v_y = lss_red.x0
v_norm = norm([v_x, v_y])

#weight matrices
Q = ComponentVector(p = 0, r = 0.1, φ = 0.15, v_x = 0/v_norm, v_y = 0.1/v_norm, β_filt = 0, ail_v = 0, ail_p = 0, rud_v = 0, rud_p = 0, ξ_β = 0.001) |> diagm
R = FlightControl.ULat(aileron_cmd = 0.1, rudder_cmd = 0.05) |> diagm

#compute gain matrix
C_aug = lqr(P_aug, Q, R)

# quickly construct the closed-loop augmented system to check the resulting
# eigenvalues, we'll do the connections properly later
F_aug_cl = F_aug - G_aug * C_aug
P_aug_cl = ss(F_aug_cl, G_aug, Hx_aug, Hu_aug)

dampreport(P_aug)
dampreport(P_aug_cl)

UndefVarError: UndefVarError: `ULat` not defined

In [20]:
A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]

#extract system state and integrator blocks from the feedback matrix
C_x = C_aug[:, 1:n_x]
C_ξ = C_aug[:, n_x+1:end]

C_fbk = C_x
C_fwd = B_22 + C_x * B_12
C_int = ComponentMatrix(zeros(n_u, n_z), Axis(u_labels), Axis(z_labels))
C_int[:, :β] .= C_ξ

#some useful signal labels
u_labels_fbk = Symbol.(string.(u_labels) .* "_fbk")
u_labels_fwd = Symbol.(string.(u_labels) .* "_fwd")
u_labels_sum = Symbol.(string.(u_labels) .* "_sum")
u_labels_int_u = Symbol.(string.(u_labels) .* "_int_u")
u_labels_int = Symbol.(string.(u_labels) .* "_int")
u_labels_ξ = Symbol.(string.(u_labels) .* "_ξ")

z_labels_sp = Symbol.(string.(z_labels) .* "_sp")
z_labels_sp1 = Symbol.(string.(z_labels) .* "_sp1")
z_labels_sp2 = Symbol.(string.(z_labels) .* "_sp2")
z_labels_err = Symbol.(string.(z_labels) .* "_err")
z_labels_sum = Symbol.(string.(z_labels) .* "_sum")
z_labels_sp_fwd = Symbol.(string.(z_labels) .* "_sp_fwd")
z_labels_sp_sum = Symbol.(string.(z_labels) .* "_sp_sum")

#the integral gain matrix is constructed by padding C_ξ with zeros in those
#columns corresponding to command variables without integral compensation
C_fbk_ss = named_ss(ss(C_fbk), u = x_labels, y = u_labels_fbk)
C_fwd_ss = named_ss(ss(C_fwd), u = z_labels_sp_fwd, y = u_labels_fwd)
C_int_ss = named_ss(ss(C_int), u = z_labels_err, y = u_labels_int_u)

int_ss = named_ss(ss(tf(1, [1,0])) .* I(2),
                    x = u_labels_ξ,
                    u = u_labels_int_u,
                    y = u_labels_int);

φ_err_sum = sumblock("φ_err = φ_sum - φ_sp_sum")
β_err_sum = sumblock("β_err = β_sum - β_sp_sum")

aileron_cmd_sum = sumblock("aileron_cmd_sum = aileron_cmd_fwd - aileron_cmd_fbk - aileron_cmd_int")
rudder_cmd_sum = sumblock("rudder_cmd_sum = rudder_cmd_fwd - rudder_cmd_fbk - rudder_cmd_int")

φ_sp_splitter = splitter(:φ_sp, 2)
β_sp_splitter = splitter(:β_sp, 2)

connections = vcat(
    Pair.(x_labels, x_labels),
    Pair.(z_labels, z_labels_sum),
    Pair.(z_labels_sp1, z_labels_sp_sum),
    Pair.(z_labels_sp2, z_labels_sp_fwd),
    Pair.(z_labels_err, z_labels_err),
    Pair.(u_labels_sum, u_labels),
    Pair.(u_labels_fwd, u_labels_fwd),
    Pair.(u_labels_fbk, u_labels_fbk),
    Pair.(u_labels_int, u_labels_int),
    Pair.(u_labels_int_u, u_labels_int_u),
    )

display(C_fbk_ss)
display(C_fwd_ss)
display(C_int_ss)

UndefVarError: UndefVarError: `n_x` not defined

Now, after designing the LQR tracker on the reduced plant, we connect it to the full lateral dynamics to get $\psi$ and $\chi$ back for later.

In [21]:
P_φβ = connect([P_lat, int_ss, C_fwd_ss, C_fbk_ss, C_int_ss,
                φ_err_sum, β_err_sum,
                aileron_cmd_sum, rudder_cmd_sum,
                φ_sp_splitter, β_sp_splitter], connections;
                w1 = z_labels_sp, z1 = y_labels_lat)

dampreport(P_φβ)

UndefVarError: UndefVarError: `int_ss` not defined

In [22]:
φsp2φ_φβ = P_φβ[:φ, :φ_sp]
φsp2β_φβ = P_φβ[:β, :φ_sp]
φsp2ail_φβ = P_φβ[:aileron_cmd, :φ_sp]
φsp2rud_φβ = P_φβ[:rudder_cmd, :φ_sp]
step(φsp2φ_φβ, 10) |> plot
step(φsp2β_φβ, 10) |> plot!
step(φsp2ail_φβ, 10) |> plot!
step(φsp2rud_φβ, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

In [23]:
βsp2β_φβ = P_φβ[:β, :β_sp]
βsp2φ_φβ = P_φβ[:φ, :β_sp]
βsp2rud_φβ = P_φβ[:rudder_cmd, :β_sp]
βsp2ail_φβ = P_φβ[:aileron_cmd, :β_sp]
step(βsp2β_φβ, 10) |> plot
step(βsp2φ_φβ, 10) |> plot!
# step(βsp2rud_φβ, 10) |> plot!
# step(βsp2ail_φβ, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

### 1.3. Design C (Integral $\phi$ and $\beta$ Control)

In [24]:
#define command variables
z_labels = [:φ, :β]
@assert tuple(z_labels...) === propertynames(FlightControl.ZLatPhiBeta())
z_trim = lss_red.y0[z_labels]
n_z = length(z_labels)

UndefVarError: UndefVarError: `ZLatPhiBeta` not defined

In [25]:
F = lss_red.A
G = lss_red.B
Hx = lss_red.C[z_labels, :]
Hu = lss_red.D[z_labels, :]
display(Hx)
display(Hu)

#define the blocks corresponding to the subset of the command variables for
#which integral compensation is required
Hx_int = Hx[z_labels, :]
Hu_int = Hu[z_labels, :]
n_int, _ = size(Hx_int)
display(Hx_int)
display(Hu_int)

F_aug = [F zeros(n_x, n_int); Hx_int zeros(n_int, n_int)]
G_aug = [G; Hu_int]
Hx_aug = [Hx zeros(n_z, n_int)]
Hu_aug = Hu

P_aug = ss(F_aug, G_aug, Hx_aug, Hu_aug)
controllability(F_aug, G_aug)

2×10 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(p = 1, r = 2, φ = 3, v_x = 4, v_y = 5, β_filt = 6, ail_v = 7, ail_p = 8, rud_v = 9, rud_p = 10)
 0.0  0.0  1.0  0.0  0.0        0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0190216  0.0  0.0  0.0  0.0  0.0

2×2 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(aileron_cmd = 1, rudder_cmd = 2)
 0.0  0.0
 0.0  0.0

2×10 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(p = 1, r = 2, φ = 3, v_x = 4, v_y = 5, β_filt = 6, ail_v = 7, ail_p = 8, rud_v = 9, rud_p = 10)
 0.0  0.0  1.0  0.0  0.0        0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0190216  0.0  0.0  0.0  0.0  0.0

2×2 ComponentMatrix{Float64} with axes Axis(φ = 1, β = 2) × Axis(aileron_cmd = 1, rudder_cmd = 2)
 0.0  0.0
 0.0  0.0

UndefVarError: UndefVarError: `n_x` not defined

In [26]:
@unpack v_x, v_y = lss_red.x0
v_norm = norm([v_x, v_y])

#weight matrices
Q = ComponentVector(p = 0, r = 0.1, φ = 0.25, v_x = 0/v_norm, v_y = 0.1/v_norm, β_filt = 0, ail_v = 0, ail_p = 0, rud_v = 0, rud_p = 0, ξ_φ = 0.1, ξ_β = 0.001) |> diagm
R = FlightControl.ULat(aileron_cmd = 0.05, rudder_cmd = 0.05) |> diagm

#compute gain matrix
C_aug = lqr(P_aug, Q, R)

# quickly construct the closed-loop augmented system to check the resulting
# eigenvalues, we'll do the connections properly later
F_aug_cl = F_aug - G_aug * C_aug
P_aug_cl = ss(F_aug_cl, G_aug, Hx_aug, Hu_aug)

dampreport(P_aug)
dampreport(P_aug_cl)

UndefVarError: UndefVarError: `ULat` not defined

In [27]:
A = [F G; Hx Hu]
B = inv(A)
B_12 = B[1:n_x, n_x+1:end]
B_22 = B[n_x+1:end, n_x+1:end]

#extract system state and integrator blocks from the feedback matrix
C_x = C_aug[:, 1:n_x]
C_ξ = C_aug[:, n_x+1:end]

C_fbk = C_x
C_fwd = B_22 + C_x * B_12
C_int = C_ξ

#some useful signal labels
u_labels_fbk = Symbol.(string.(u_labels) .* "_fbk")
u_labels_fwd = Symbol.(string.(u_labels) .* "_fwd")
u_labels_sum = Symbol.(string.(u_labels) .* "_sum")
u_labels_int_u = Symbol.(string.(u_labels) .* "_int_u")
u_labels_int = Symbol.(string.(u_labels) .* "_int")
u_labels_ξ = Symbol.(string.(u_labels) .* "_ξ")

z_labels_sp = Symbol.(string.(z_labels) .* "_sp")
z_labels_sp1 = Symbol.(string.(z_labels) .* "_sp1")
z_labels_sp2 = Symbol.(string.(z_labels) .* "_sp2")
z_labels_err = Symbol.(string.(z_labels) .* "_err")
z_labels_sum = Symbol.(string.(z_labels) .* "_sum")
z_labels_sp_fwd = Symbol.(string.(z_labels) .* "_sp_fwd")
z_labels_sp_sum = Symbol.(string.(z_labels) .* "_sp_sum")

#the integral gain matrix is constructed by padding C_ξ with zeros in those
#columns corresponding to command variables without integral compensation
C_fbk_ss = named_ss(ss(C_fbk), u = x_labels, y = u_labels_fbk)
C_fwd_ss = named_ss(ss(C_fwd), u = z_labels_sp_fwd, y = u_labels_fwd)
C_int_ss = named_ss(ss(C_int), u = z_labels_err, y = u_labels_int_u)

int_ss = named_ss(ss(tf(1, [1,0])) .* I(2),
                    x = u_labels_ξ,
                    u = u_labels_int_u,
                    y = u_labels_int);

φ_err_sum = sumblock("φ_err = φ_sum - φ_sp_sum")
β_err_sum = sumblock("β_err = β_sum - β_sp_sum")

aileron_cmd_sum = sumblock("aileron_cmd_sum = aileron_cmd_fwd - aileron_cmd_fbk - aileron_cmd_int")
rudder_cmd_sum = sumblock("rudder_cmd_sum = rudder_cmd_fwd - rudder_cmd_fbk - rudder_cmd_int")

φ_sp_splitter = splitter(:φ_sp, 2)
β_sp_splitter = splitter(:β_sp, 2)

connections = vcat(
    Pair.(x_labels, x_labels),
    Pair.(z_labels, z_labels_sum),
    Pair.(z_labels_sp1, z_labels_sp_sum),
    Pair.(z_labels_sp2, z_labels_sp_fwd),
    Pair.(z_labels_err, z_labels_err),
    Pair.(u_labels_sum, u_labels),
    Pair.(u_labels_fwd, u_labels_fwd),
    Pair.(u_labels_fbk, u_labels_fbk),
    Pair.(u_labels_int, u_labels_int),
    Pair.(u_labels_int_u, u_labels_int_u),
    )

display(C_fbk_ss)
display(C_fwd_ss)
display(C_int_ss)

UndefVarError: UndefVarError: `n_x` not defined

Now, after designing the LQR tracker on the reduced plant, we connect it to the full lateral dynamics to get $\psi$ and $\chi$ back for later.

In [28]:
P_φβ = connect([P_lat, int_ss, C_fwd_ss, C_fbk_ss, C_int_ss,
                φ_err_sum, β_err_sum,
                aileron_cmd_sum, rudder_cmd_sum,
                φ_sp_splitter, β_sp_splitter], connections;
                w1 = z_labels_sp, z1 = y_labels_lat)

dampreport(P_φβ)

UndefVarError: UndefVarError: `int_ss` not defined

In [29]:
φsp2φ_φβ = P_φβ[:φ, :φ_sp]
φsp2β_φβ = P_φβ[:β, :φ_sp]
φsp2ail_φβ = P_φβ[:aileron_cmd, :φ_sp]
φsp2rud_φβ = P_φβ[:rudder_cmd, :φ_sp]
step(φsp2φ_φβ, 10) |> plot
step(φsp2β_φβ, 10) |> plot!
step(φsp2ail_φβ, 10) |> plot!
step(φsp2rud_φβ, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

In [30]:
βsp2β_φβ = P_φβ[:β, :β_sp]
βsp2φ_φβ = P_φβ[:φ, :β_sp]
βsp2rud_φβ = P_φβ[:rudder_cmd, :β_sp]
βsp2ail_φβ = P_φβ[:aileron_cmd, :β_sp]
step(βsp2β_φβ, 10) |> plot
step(βsp2φ_φβ, 10) |> plot!
# step(βsp2rud_φβ, 10) |> plot!
# step(βsp2ail_φβ, 10) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

## 2. Roll Rate Control Augmentation

We now design a roll rate CAS on top of the plant with $\phi$ and $\beta$ tracking. This will be a SISO loop connected to the $\phi_{sp}$ input. That is, we will provide roll rate tracking via the bank angle set point. Clearly, one or more integrators will be needed.

In [31]:
P_φ2p = P_φβ[:p, :φ_sp];

UndefVarError: UndefVarError: `P_φβ` not defined

In [32]:
bodeplot(P_φ2p)

UndefVarError: UndefVarError: `P_φ2p` not defined

In [33]:
nyquistplot(P_φ2p; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_φ2p` not defined

In [34]:
p2φ_int = tf(1, [1, 0]) |> ss
P_p2φ_opt = series(p2φ_int, ss(P_φ2p))

t_sim_p2φ = 10
lower_bounds = PIDParams(; k_p = 0.1, k_i = 0.0, k_d = 0.0, τ_f = 0.01)
upper_bounds = PIDParams(; k_p = 10.0, k_i = 35.0, k_d = 1.5, τ_f = 0.01)
settings = Settings(; t_sim = t_sim_p2φ, lower_bounds, upper_bounds)
weights = Metrics(; Ms = 1, ∫e = 10, ef = 2, ∫u = 0.1, up = 0.00)
params_0 = PIDParams(; k_p = 1.5, k_i = 3, k_d = 0.1, τ_f = 0.01)

p2φ_results = optimize_PID(P_p2φ_opt; params_0, settings, weights, global_search)

@unpack k_p, k_i, k_d, T_i, T_d = p2φ_results.params
@unpack k_p, k_i, k_d, T_i, T_d = p2φ_results.params
@show k_p, k_i, k_d, T_i, T_d
@show p2φ_results.metrics
@show p2φ_results.exit_flag

UndefVarError: UndefVarError: `P_φ2p` not defined

In [35]:
p2φ_PID = build_PID(p2φ_results.params)
C_p2φ = named_ss(series(p2φ_int, p2φ_PID), :C_p2φ; u = :p_err, y = :φ_sp)

UndefVarError: UndefVarError: `p2φ_results` not defined

In [36]:
#steady-state error for a unit step input, now zero
L_p2φ = series(C_p2φ, P_φ2p)

T_p2φ = output_comp_sensitivity(P_φ2p, C_p2φ) #this is the closed loop transfer function
T_p2φ_step = step(T_p2φ, t_sim_p2φ)
stepinfo(T_p2φ_step) |> display
T_p2φ_step |> plot

UndefVarError: UndefVarError: `C_p2φ` not defined

In [37]:
marginplot(L_p2φ)

UndefVarError: UndefVarError: `L_p2φ` not defined

In [38]:
gangoffourplot(P_φ2p, C_p2φ)

UndefVarError: UndefVarError: `P_φ2p` not defined

In [39]:
p2φ_sum = sumblock("p_err = p_sp - p")
P_pβ = connect([P_φβ, p2φ_sum, C_p2φ], [:p_err=>:p_err, :p=>:p, :φ_sp=>:φ_sp], w1 = [:p_sp, :β_sp], z1 = P_φβ.y)

#check we get the same response as with the SISO closed loop
psp2p = P_pβ[:p, :p_sp]
psp2β = P_pβ[:β, :p_sp]
psp2a = P_pβ[:aileron_cmd, :p_sp]
psp2r = P_pβ[:rudder_cmd, :p_sp]
step(psp2p, t_sim_p2φ) |> plot
step(psp2β, t_sim_p2φ) |> plot!
step(psp2a, t_sim_p2φ) |> plot!
step(psp2r, t_sim_p2φ) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

To avoid transients on mode changes, we want the initial output of the roll rate loop to match the current value of the bank angle setpoint at the roll-yaw SAS input, $\phi_{sp}$.

Whenever roll rate mode is enabled after a mode change, we first reset both the integrator and the PID. Then we set PID integrator's state to the appropriate value to achieve the required $\phi_{sp}$. We have:
$$ y_{int} = 0 $$
$$ y_{pid} = k_p y_{int} + k_i x_{i0} = k_i x_{i0} = \phi_{sp} $$

Therefore:
$$x_{i0} = \phi_{sp}/k_i$$

## 3. Course Angle Tracker

We start from the plant with embedded bank angle and sideslip control.

In [40]:
P_φ2χ = P_φβ[:χ, :φ_sp];
bodeplot(P_φ2χ)
step(P_φ2χ, 10) |> plot

UndefVarError: UndefVarError: `P_φβ` not defined

In [41]:
marginplot(P_φ2χ)

UndefVarError: UndefVarError: `P_φ2χ` not defined

In [42]:
nyquistplot(P_φ2χ; unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `P_φ2χ` not defined

We can see that without compensation the loop transfer function is already type-1, that is, gain goes to infinity for zero frequency. This ensures zero steady-state error, but not in the presence of external disturbances (such as crosswind). For zero steady-state disturbances rejection we need integral compensation. The result is a type 2 open loop system.

In [43]:
t_sim_χ2φ = 30
lower_bounds = PIDParams(; k_p = 0.1, k_i = 0.3, k_d = 0.0, τ_f = 0.01)
upper_bounds = PIDParams(; k_p = 10.0, k_i = 0.3, k_d = 0.0, τ_f = 0.01)
settings = Settings(; t_sim = t_sim_χ2φ, lower_bounds, upper_bounds)
weights = Metrics(; Ms = 3, ∫e = 10, ef = 1, ∫u = 0.00, up = 0.01)
params_0 = PIDParams(; k_p = 3., k_i = 0.3, k_d = 0.0, τ_f = 0.01)

χ2φ_results = optimize_PID(P_φ2χ; params_0, settings, weights, global_search)
χ2φ_PID = build_PID(χ2φ_results.params)
C_χ2φ = named_ss(χ2φ_PID, :C_χ2φ; u = :χ_err, y = :φ_sp);

@unpack k_p, k_i, k_d, T_i, T_d = χ2φ_results.params
@show k_p, k_i, k_d, T_i, T_d
@show χ2φ_results.metrics
@show χ2φ_results.exit_flag

UndefVarError: UndefVarError: `P_φ2χ` not defined

In [44]:
L_χ2φ = series(C_χ2φ, P_φ2χ)
nyquistplot(L_χ2φ, unit_circle = true, Ms_circles = [1.0])
plot!(ylims = (-2,2), xlims = (-2,2))

UndefVarError: UndefVarError: `C_χ2φ` not defined

In [45]:
marginplot(L_χ2φ)

UndefVarError: UndefVarError: `L_χ2φ` not defined

Here we are particularly interested in the load disturbance to output transfer function P/(1+C). In order for the controller to be able to counter the effects of crosswind, this function needs to show low frequency roll-off. And indeed, forcing a nonzero integral term in the PID achieves this goal.

In [46]:
gangoffourplot(P_φ2χ, C_χ2φ)

UndefVarError: UndefVarError: `P_φ2χ` not defined

In [47]:
T_χ2φ = output_comp_sensitivity(P_φ2χ, C_χ2φ) #input to output response
T_χ2φ_step = step(T_χ2φ, 2t_sim_χ2φ)
stepinfo(T_χ2φ_step) |> display
T_χ2φ_step |> plot

UndefVarError: UndefVarError: `P_φ2χ` not defined

The drawback of the forced inclusion of integral action is an undesirably large overshoot. However, for the step response it is also important to consider the input to control signal response, because obviously our bank angle demand will be limited. 

And in this case, saturation will work in our favor. For large initial step responses, it will halt integration before the integrator's state reaches the values predicted by the linear model. Therefore, the actual overshoot will be smaller.

In [48]:
CS_χ2φ = G_CS(P_φ2χ, C_χ2φ) #input to control signal response
CS_χ2φ_step = step(CS_χ2φ, t_sim_χ2φ)
CS_χ2φ_step |> plot!

UndefVarError: UndefVarError: `P_φ2χ` not defined

In [49]:
χ2φ_sum = sumblock("χ_err = χ_sp - χ")
P_χβ = connect([P_φβ, χ2φ_sum, C_χ2φ], [:χ_err=>:χ_err, :χ=>:χ, :φ_sp=>:φ_sp], w1 = [:χ_sp, :β_sp], z1 = P_φβ.y)

#check we get the same response as with the SISO closed loop
χsp2χ = P_χβ[:χ, :χ_sp]
χsp2β = P_χβ[:β, :χ_sp]
χsp2a = P_χβ[:aileron_cmd, :χ_sp]
step(χsp2χ, t_sim_χ2φ) |> plot
step(χsp2β, t_sim_χ2φ) |> plot!
step(χsp2a, t_sim_χ2φ) |> plot!

UndefVarError: UndefVarError: `P_φβ` not defined

To avoid transients on mode changes, we want the initial output of the course angle loop to match the current value of the bank angle setpoint at the roll-yaw SAS input, $\phi_{sp}$.

Whenever $\chi2\phi$ is enabled after a mode change, we reset the PID. Then we set the PID integrator's state to the appropriate value to achieve the required $\phi$. We have:
$$ y_{pid} = k_p (\chi_{sp} - \chi) + k_i x_{i0} = k_i x_{i0} $$
$$ \phi_{sp} = y_{pid}$$

Therefore:
$$x_{i0} = y_{pid} / k_i = \phi_{sp}/k_i = \phi/k_i$$